<a href="https://colab.research.google.com/github/nikhilvkth/computer_vision/blob/main/CV_image_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Load YOLO model and classes
net = cv2.dnn.readNet('/Users/nikhilvalsankulangareth/darknet/yolov3.weights',
                      '/Users/nikhilvalsankulangareth/darknet/cfg/yolov3.cfg')
classes = []

with open('/Users/nikhilvalsankulangareth/darknet/data/coco.names', 'r') as f:
    classes = [line.strip() for line in f]

layer_names = net.getUnconnectedOutLayersNames()

# Function to perform image detection using YOLO
def detect_objects_yolo(image_path):
    # Read the image
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Get information about detected objects
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Apply non-maximum suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Collect labels of detected objects
    detected_objects = [str(classes[class_ids[i]]) for i in indices]

    # Draw rectangles around the detected objects
    for i in indices:
        i = i.item()  # Convert to scalar
        box = boxes[i]
        x, y, w, h = box
        label = str(classes[class_ids[i]])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return img, detected_objects

# Specify the path to your image
image_path = '/Users/nikhilvalsankulangareth/Downloads/cat_2.jpg'

# Perform object detection using YOLO
result_image_yolo, detected_objects = detect_objects_yolo(image_path)

# Display the original and result images using matplotlib
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

# Result image with detected objects using YOLO
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(result_image_yolo, cv2.COLOR_BGR2RGB))
plt.title('Image with Detected Objects (YOLO)')
plt.axis('off')

# Print the detected objects
print("Detected Objects:", detected_objects)

plt.show()
